# Scrape products data from DIA website

## Load Libraries

In [5]:
from bs4 import BeautifulSoup
import requests

## Get sample HTML from DIA website

In [13]:
data_jamon = requests.get('https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/c/L2001')

if data_jamon.status_code == 200:
    print("OK")
else:
    raise Exception("Error en la petición")

html_jamon = data_jamon.text
print(html_jamon[:1000])

OK
<!DOCTYPE html>
    <html lang="es">
      <head>
        <link rel="stylesheet" type="text/css" href="/plp-front/assets/1-24-0/style.css">
        <link rel="preload" href="/plp-front/assets/1-24-0/roboto-medium-webfont.woff2" as="font" type="font/woff2" crossorigin>
        <link rel="preload" href="/plp-front/assets/1-24-0/roboto-medium-webfont.woff" as="font" type="font/woff" crossorigin>
        <link rel="preload" href="/plp-front/assets/1-24-0/Roboto-Medium.ttf" as="font" type="font/ttf" crossorigin>
        <link rel="preload" href="/plp-front/assets/1-24-0/roboto-bold-webfont.woff2" as="font" type="font/woff2" crossorigin>
        <link rel="preload" href="/plp-front/assets/1-24-0/roboto-bold-webfont.woff" as="font" type="font/woff" crossorigin>
        <link rel="preload" href="/plp-front/assets/1-24-0/Roboto-Bold.ttf" as="font" type="font/ttf" crossorigin>
        <link rel="preload" href="/plp-front/assets/1-24-0/Roboto-Black.ttf" as="font" type="font/ttf" crossorigin>
 

In [15]:
# Create the soup
soup_jamon = BeautifulSoup(html_jamon)
print(type(soup_jamon))

<class 'bs4.BeautifulSoup'>


In [22]:
for content in soup_jamon.body.contents:
    print(content)

len([x for x in soup_jamon.body.contents if x!= "\n"])



<div id="app"><div class="layout"><div class="content"><!--[--><div class="plp-view" data-test-id="plp-view" data-v-bf0fd08a=""><div class="plp-view__header-container" data-test-id="header-component" data-v-bf0fd08a=""><!--[--><div class="dia-header" data-test-id="dia-header-component" data-v-969db9f0=""><div class="dia-header__section dia-header__section--start" data-v-969db9f0=""><a class="header-logo dia-header__logo" data-test-id="header-logo" data-v-969db9f0="" data-v-a0b5f86b="" href="/" rel="external"><img alt="LogoDia" class="header-logo__image" data-v-a0b5f86b="" height="31" src="/plp-front/assets/1-24-0/logo-dia.svg" width="56"/></a><div class="dia-header__left-icons" data-v-969db9f0=""><!--[--><div class="category-button__mobile-container"><button aria-label="categories list menú button" class="category-button__mobile dia-icon-dehaze" data-test-id="mobile-category-button"></button></div><button aria-label="dissabled categories list menú button" class="category-button__disa

18

In [42]:
script_tag = soup_jamon.body.find_all("script", id="vike_pageContext")[0]
script_tag = str(script_tag.string)

import json
d_products_jamon = json.loads(script_tag)
d_products_jamon.keys()

dict_keys(['pageProps', 'routeParams', 'INITIAL_STATE', 'endpoints', 'envVariables', 'finalHeaders', 'serverOriginalUrl', 'abortReason', '_urlRewrite', '_urlRedirect', 'abortStatusCode', '_abortCall', '_pageContextInitHasClientData', '_pageId'])

In [50]:
len(d_products_jamon["INITIAL_STATE"]["plp_items"])

21

In [54]:
d_products_jamon["INITIAL_STATE"]["plp_items"][:-1]

[{'brand': 'Nuestra Alacena',
  'display_name': 'Pechuga de pavo Nuestra Alacena sobre 2 x 200 g',
  'expiration_days': 0,
  'image': '/product_images/273810/273810_ISO_0_ES.jpg',
  'object_id': '273810',
  'prices': {'currency': 'EUR',
   'discount_percentage': 0,
   'is_club_price': False,
   'is_promo_price': False,
   'measure_unit': 'KILO',
   'price': 3.59,
   'price_per_unit': 8.98,
   'strikethrough_price': 3.59},
  'product_info': 'Sin gluten | Sin lactosa',
  'sku_id': '273810',
  'units_in_cart': 0,
  'units_in_stock': 35,
  'url': '/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/p/273810',
  'weight': 0},
 {'brand': 'Nuestra Alacena',
  'display_name': 'Jamón cocido extra Nuestra Alacena sobre 2 x 225 g',
  'expiration_days': 0,
  'image': '/product_images/273957/273957_ISO_0_ES.jpg',
  'object_id': '273957',
  'prices': {'currency': 'EUR',
   'discount_percentage': 0,
   'is_club_price': False,
   'is_promo_price': False,
   'measure_unit': 'KILO',
   'price'

## Get products and Categories from Menu

In [67]:
def get_html(url):
    try:
        data = requests.get(url)
        html = data.text         
        return html
    except:
        print("URL NOT FOUND")


In [69]:
url_base = "https://www.dia.es/"

get_html(url=url_base)

'<!DOCTYPE html>\n  \n    <html lang="es">\n      <head>\n        <link rel="stylesheet" type="text/css" href="/home-front/assets/1-20-0/style.css">\n        <link rel="preload" href="/home-front/assets/1-20-0/roboto-medium-webfont.woff2" as="font" type="font/woff2" crossorigin>\n        <link rel="preload" href="/home-front/assets/1-20-0/roboto-medium-webfont.woff" as="font" type="font/woff" crossorigin>\n        <link rel="preload" href="/home-front/assets/1-20-0/Roboto-Medium.ttf" as="font" type="font/ttf" crossorigin>\n        <link rel="preload" href="/home-front/assets/1-20-0/roboto-bold-webfont.woff2" as="font" type="font/woff2" crossorigin>\n        <link rel="preload" href="/home-front/assets/1-20-0/roboto-bold-webfont.woff" as="font" type="font/woff" crossorigin>\n        <link rel="preload" href="/home-front/assets/1-20-0/Roboto-Bold.ttf" as="font" type="font/ttf" crossorigin>\n        <link rel="preload" href="/home-front/assets/1-20-0/Roboto-Black.ttf" as="font" type="font